# Работа с xml и html

Будем использовать пакет lxml (http://lxml.de/)

In [1]:
import lxml.html
import lxml.etree
import requests

Пусть есть некоторый xml.

In [2]:
our_xml = """
                <root>
                    <a>
                        text1
                        <d q='d1'>
                          text d1
                        </d>
                        <d q='d2'>
                          text d2
                        </d>
                    </a>
                    
                    <b q='7'>                    
                        text b
                    </b>
                    
                    <b q='5'>
                        text 3
                        <d q='d3'>
                           text d3
                        </d>
                    </b>
                    
                </root>
            """

In [3]:
tree = lxml.etree.fromstring(our_xml)

Получаем дерево

In [4]:
for child in tree.getchildren():
    print(child.tag, child.attrib)

a {}
b {'q': '7'}
b {'q': '5'}


Удобные методы для XPath запросов:

Все узлы с именем `<d>`

In [5]:
tree.xpath('//d')

[<Element d at 0x7f69506383c8>,
 <Element d at 0x7f6940a7da48>,
 <Element d at 0x7f6940a7d308>]

Узел `<b>`, который является потомком узкла `<root>` и имеет аттрибут `q` равный 7

In [6]:
tree.xpath('/root/b[@q="7"]')

[<Element b at 0x7f6950638708>]

Значения атрибута `q` у всех узлов `<d>` которые являются потомками всех узлов `<a>`

In [7]:
tree.xpath('//a//d/@q')

['d1', 'd2']

Взять текстовый контент узла `<d>` с атрибутом `q` равным `d1`, который является прямым потомком узла `<a>`, который является прямым потомком корневого элемента. 

In [8]:
tree.xpath('/root/a/d[@q="d1"]/text()')

['\n                          text d1\n                        ']

Когда xml большой (дамп Википедии), на помощь приходит SAX-like парсеры

In [9]:
import xml.sax
import io

class MyContentHandler(xml.sax.ContentHandler):
    def __init__(self):
        self.level = 0
    
    def startElement(self, name, attributes):
        self.level += 1
        print('  ' * self.level, 'Start <%s>' % name, 'attr:', attributes.items())        
        
    def endElement(self, name):
        print('  ' * self.level, 'End </%s>' % name)
        self.level -= 1

    def characters(self, data):
        text = data.strip()
        if len(text) > 0:
            print('  ' * self.level, ' Text:', text)
        
handler = MyContentHandler()
parser = xml.sax.make_parser()
parser.setContentHandler(handler)
parser.parse(io.StringIO(our_xml))
#parser.parse(open('data/wiki.xml'))

   Start <root> attr: []
     Start <a> attr: []
      Text: text1
       Start <d> attr: [('q', 'd1')]
        Text: text d1
       End </d>
       Start <d> attr: [('q', 'd2')]
        Text: text d2
       End </d>
     End </a>
     Start <b> attr: [('q', '7')]
      Text: text b
     End </b>
     Start <b> attr: [('q', '5')]
      Text: text 3
       Start <d> attr: [('q', 'd3')]
        Text: text d3
       End </d>
     End </b>
   End </root>


C HTML можно работать аналогичным образом, lxml пытается привести его к нормальному виду (добавить необходимые по спецификации теги, исправить ошибки и пр.)

In [10]:
response = requests.get('http://fontanka.ru')
response.status_code

200

In [11]:
print(response.text.strip()[:120])

<!DOCTYPE html>
<html>
<head>
	<meta charset="windows-1251"/>

	<meta name="Keywords" content="Санкт-Петербург, Петербур


In [12]:
tree = lxml.html.fromstring(response.text)

In [13]:
tree.xpath('//title/text()')

['Новости Санкт-Петербурга, последние новости дня, новости бизнеса - Фонтанка.ру']

In [14]:
tree.xpath('//div[contains(@class, "common_news")]//div[@class="entry_title"]/a/text()')

['Фонтан на Шлиссельбургском ликвидировали, но его будут поминать 22 дома',
 'Буксиры станцевали у Английской набережной',
 'Ладожский мост вернули водителям',
 'Ефремов и Орлуша матом объяснили, зачем материть зрителей',
 'Поджог «Мерседеса» в Приморском районе зафиксировали камеры наблюдения',
 'В Петербурге будильник перепутали с бомбой',
 'Дети погибли при квартирном пожаре в Подмосковье',
 'В Петербурге за дачу взятки полицейским арестована подруга Анны Семенович',
 'The Sunday Times сообщила о попытках РФ повлиять на выборы в Британии',
 '«Фонтанка» ищет корреспондента в рубрику «Бизнес»',
 'В Вашингтоне пропал дуб, который Трампу привез друг из Франции',
 'Ядовитые гусеницы готовятся к атаке на Лондон',
 'Ничья с ЦСКА уменьшила шансы «Зенита» на выход в Лигу чемпионов',
 'Албин отфутболил «Зениту» вопросы про сломанное табло',
 '«Удельную» проверили за час',
 'Петербуржцы выстроились в очередь за арктической мощью',
 'Павел Дуров позвал москвичей на митинг за свободу Интернета',